In [1]:
import pandas as pd
test_file_path='~/Downloads/test 2.csv'
test_dataset=pd.read_csv(test_file_path)
# use first 50 rows for dataset
test_dataset = test_dataset.head(50)

In [1]:
import pandas as pd
val_file_path='~/Downloads/validation.csv'
val_dataset=pd.read_csv(val_file_path)
# use first 50 rows for dataset
gt_dataset = val_dataset.head(10)
val_dataset = val_dataset.tail(40)

In [2]:
# construct train sets
# load train.csv
import dspy
train_file_path='./train.csv'
train_dataset=pd.read_csv(train_file_path)
l=len(train_dataset)
trainset=[]
for i in range(l):
    trainset.append(dspy.Example(document=train_dataset.iloc[i]['documents'], highlights=train_dataset.iloc[i]['summaries'], length=50, style="formal", context="").with_inputs("document","length","style","context"))

/Users/meguru_kiss/anaconda3/envs/textsum/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dspy.teleprompt import BootstrapFewShot
from dspy_utils import metric, PersonalizedSummary
summarizer=dspy.ChainOfThought(PersonalizedSummary)
teleprompt = BootstrapFewShot(metric=metric)
compiled_predictor=teleprompt.compile(summarizer, trainset=trainset)

 40%|████      | 4/10 [00:14<00:22,  3.67s/it]

Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.


In [6]:
compiled_predictor(document=val_dataset.iloc[0]['article'],length=50,style='formal',context="")

Prediction(
    reasoning="The document discusses BAE Systems' innovative Broadsword range, which features the Spine vest designed for military use. This vest incorporates e-textiles for wireless energy transfer and charging of various military equipment, monitored via a smartphone app. The summary highlights the technological advancements and potential applications of this equipment beyond military use.",
    summary='BAE Systems has developed the Broadsword range of devices, centered around the Spine vest, which utilizes e-textiles for wireless charging of military equipment. The vest can power multiple devices simultaneously and integrates with augmented reality technology, with applications extending to fire and rescue services and police forces.'
)

In [7]:
dspy.inspect_history(n=1)





[2024-12-07T18:20:35.690020]

System message:

Your input fields are:
1. `document` (str): The document to summarize
2. `length` (int): The maximum length of the summary
3. `style` (str): The style of the summary, like 'formal', 'casual'
4. `context` (str): May contain relevant facts

Your output fields are:
1. `reasoning` (str)
2. `summary` (str): The summary of the document

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## document ## ]]
{document}

[[ ## length ## ]]
{length}

[[ ## style ## ]]
{style}

[[ ## context ## ]]
{context}

[[ ## reasoning ## ]]
{reasoning}

[[ ## summary ## ]]
{summary}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `document`, `length`, `style`, `context`, produce the fields `summary`.


User message:

[[ ## document ## ]]
Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such a

In [8]:
valset=[]
for i in range(l//2):
    valset.append(dspy.Example(document=val_dataset.iloc[i]['article'], highlights=val_dataset.iloc[i]['highlights'], length=50, style="formal", context="").with_inputs("document","length","style","context"))
for i in range(l//2,l):
    valset.append(dspy.Example(document=val_dataset.iloc[i]['article'], highlights=val_dataset.iloc[i]['highlights'], length=50, style="casual", context="").with_inputs("document","length","style","context"))
evaluate=dspy.Evaluate(devset=valset,metric=metric)
evaluate(compiled_predictor)

2024/12/07 15:45:04 INFO dspy.evaluate.evaluate: Average Metric: 50.0 / 10 (500.0%)


500.0

In [8]:
compiled_predictor.save("./compiled_predictor_bootstrapfewshots")

In [9]:
summarizer=dspy.ChainOfThought(PersonalizedSummary)
summarizer.load("./compiled_predictor_bootstrapfewshots")


In [10]:
summarizer(document=val_dataset.iloc[0]['article'],length=50,style='formal',context="")

Prediction(
    reasoning="The document highlights the innovative advancements in military technology through BAE Systems' Broadsword range, focusing on the Spine vest that integrates e-textiles for powering and charging devices. It discusses the functionalities of the Spine, including its ability to wirelessly charge equipment and monitor energy usage via a smartphone app. The summary captures the key features and potential applications of this technology within military and emergency service contexts.",
    summary='BAE Systems has developed the Broadsword range, featuring the Spine vest that utilizes e-textiles to wirelessly charge military equipment. The vest can power multiple devices simultaneously and monitor energy use through a smartphone app, making it a versatile tool for soldiers. This technology may also benefit emergency services.'
)

In [11]:
dspy.inspect_history(n=1)





[2024-12-07T18:21:35.795732]

System message:

Your input fields are:
1. `document` (str): The document to summarize
2. `length` (int): The maximum length of the summary
3. `style` (str): The style of the summary, like 'formal', 'casual'
4. `context` (str): May contain relevant facts

Your output fields are:
1. `reasoning` (str)
2. `summary` (str): The summary of the document

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## document ## ]]
{document}

[[ ## length ## ]]
{length}

[[ ## style ## ]]
{style}

[[ ## context ## ]]
{context}

[[ ## reasoning ## ]]
{reasoning}

[[ ## summary ## ]]
{summary}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `document`, `length`, `style`, `context`, produce the fields `summary`.


User message:

[[ ## document ## ]]
Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such a